<a href="https://colab.research.google.com/github/OVP2023/NN/blob/main/DZ_RNN2_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)
и задавались следующим образом:
x - последовательность цифр
y(0) = x(0), y(i) = x(i) + x(0). Если y(i) >= 10, то y(i) = y(i) - 10

Задача:
научить модель предсказывать y(i) по x(i)
пробовать RNN, LSTM, GRU'''

In [ ]:
import torch
from torch import nn
import re
import random
import tqdm
import time

In [ ]:
MAX_LEN=40 #Вводим максимальную длину последовательности

In [ ]:
x=[] #формируем последовательность x
for j in range(random.randint(1,MAX_LEN)):
    x.append(random.randint(0,9))
print('x',x,len(x))
y=[] #формируем последовательность y
y_vr=0
y.append(x[0])
for i in range(1,len(x)):
    y_vr = x[i]+x[0]
    if (y_vr < 10):
        y.append(y_vr)
    else:
        y.append(y_vr-10)

print('y',y,len(y))


x [5, 1, 3, 7, 8, 8, 3, 8, 8, 4, 8, 6, 0, 8, 8, 6, 7, 2, 4, 8, 8, 0, 4, 4, 1, 4, 8, 1, 5, 0, 1, 9, 5, 4, 7, 4, 1] 37
y [5, 6, 8, 2, 3, 3, 8, 3, 3, 9, 3, 1, 5, 3, 3, 1, 2, 7, 9, 3, 3, 5, 9, 9, 6, 9, 3, 6, 0, 5, 6, 4, 0, 9, 2, 9, 6] 37


In [ ]:
# Модель нейронной сети
class DecryptModel(nn.Module):
    def __init__(self):
        super(DecryptModel, self).__init__()
        self.embedding = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
        self.lstm = nn.LSTM(
            input_size=EMBEDDING_DIM,
            hidden_size=LSTM_UNITS,
            num_layers=1,
            batch_first=True,
            bidirectional=False
        )
        self.fc = nn.Linear(LSTM_UNITS, VOCAB_SIZE)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out)
        return output

In [ ]:
# Инициализация модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DecryptModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
for epoch in range(NUM_EPOCHS):
   total_loss = 0
   for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

            # Изменяем форму для вычисления потерь
        loss = criterion(outputs.view(-1, VOCAB_SIZE), targets.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if i % 50 == 0:
            print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Step [{i}/{len(train_loader)}], Loss: {loss.item():.4f}')

print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Average Loss: {total_loss/len(train_loader):.4f}')